# 

10 km res
A3 and A4

2019 (start) - 2024 (end)
- calculate yearly average change

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import torch
import matplotlib.pyplot as plt

In [14]:
path = "/home/kim/data/icesat2/ATL15_A4_0325_01km_004_05.nc"
path = "/home/kim/data/icesat2/ATL15_A3_0325_01km_004_05.nc"

path = "/home/kim/data/icesat2/ATL15_A4_0325_01km_004_05.nc"
path = "/home/kim/data/icesat2/ATL15_A3_0325_10km_004_05.nc"

ds = xr.load_dataset(path)

In [15]:
ds

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes: (12/51)
    GDAL_AREA_OR_POINT:                 Area
    Conventions:                        CF-1.7
    short_name:                         ATL15
    level:                              L3B
    title:                              SET_BY_META
    description:                        This data set (ATL15) contains season...
    ...                                 ...
    processing_level:                   3B
    references:                         http://nsidc.org/data/icesat2/data.html
    project:                            ICESat-2 > Ice, Cloud, and land Eleva...
    instrument:                         ATLAS > Advanced Topographic Laser Al...
    platform:                           ICESat-2 > Ice, Cloud, and land Eleva...
    source:                             Spacecraft